## Chapter 4: Bond pricing with a flat term structure

In this section we use the present value framework of the previous chapter to price bonds and other fixed income securities. What distinguishes bonds is that the future payments are set when the security is issued. The simplest, and most typical bond, is a fixed interest, constant maturity bond with no default risk. There is however a large number of alternative contractual features of bonds. The bond could for example ba an annuity bond, paying a fixed amount each period. For such a bond the principal amount outstanding is paid gradually during the life of the bond. The interest rate the bond pays need not be fixed, it could be a floating rate, the interest rate paid could be a function of some market rate. Many bonds are issued by corporations, and in such cases there is a risk that the company issued the bond defaults, and the bond does not pay the complete promised amount. Another thing that makes bond pricing difficult in practice, is that interest rates tend to change over time.


We start by assuming that all the promised payments are certain. Then the bond current price $B_0$ is found as the present value of these payments. The first step of pricing is to use the terms of the bond to find the promised payments. We start by considering a fixed  interest bond with no default risk. Such bonds are typically bonds issued by governments. The bond is a promis to pay a face value $F$ at the maturity date $T$ periods from now. Each period the bond pays a fixed percentage amount of the face value as coupon C. The cash flows from the bond thus look as follows.

In general a bond price is found as the present value

$$B_0=d_1C_1+d_2C_2+\cdot\cdot\cdot+d_TC_T=\sum_{t=1}^{T}d_tC_t$$

where $d_t$ is the discount factor, or the time 0 price of a payment of 1 at time $t$. To fully specify the
problem it is necessary to find all discount factors $d_t$. In this chapter we will work with a specially simple
specifiction of the term structure, namely that it is flat, and specified by the interest rate $r$.

### 4.1 Flat term structure with discrete, annual compounding
This is the simplest possible specification of a term structure,

$$d_t = \Big(\frac{1}{1+r}\Big)^t=\frac{1}{(1+r)^t}$$

### 4.1.1 Bond Price
The current bond price $(B_0)$ is the present value of the cash flows from the bond

$$B_0=\sum_{t=1}^{T}\Bigg(\frac{1}{(1+r)^t}\Bigg)C_t=\sum_{t=1}^{T}\frac{C_t}{(1+r)^t}$$

If we continue with the example of a standard fixed interest bond, where $C_t = C$ when $t < T$ and
$CT = C + F$,

#### Example
A 3 year bond with a face value of $100 makes annual coupon payments of 10%. The current interest rate
(with annual compounding) is 9%.

1. Determine the current bond price.

The current bond price: $B_0=\frac{10}{(1+0.09)^1}+\frac{10}{(1+0.09)^2}+\frac{110}{(1+0.09)^3}=102.531$

In [1]:
# Code 4.1: Bond pricing calculation with discrete, annual compoudning

def bonds_price_discrete(times, cashflows, r):
    
    p = 0
    
    for i in range(len(times)):
        p += cashflows[i] / np.power((1 + r), times[i])
        
    return p

In [2]:
import numpy as np 

c = np.array([10, 10, 110])
t = np.arange(1, 4)
r = 0.09
d = (1. / np.power((1 + r), t))
B = np.sum(d * c)

In [3]:
print('bonds price = {:.3f}'.format(bonds_price_discrete(t, c, r)))

bonds price = 102.531


### 4.1.1 Yield to maturity
Since bonds are issued in terms of interest rate, it is also useful to find an interest rate number that
summarizes the terms of the bond. The obvious way of doing that is asking the question: What is the
internal rate of return on the investment of buying the bond now and keeping the bond to maturity?
The answer to that question is the yield to maturity of a bond. The yield to maturity is the interest
rate that makes the present value of the future coupon payments equal to the current bond price, that
is, for a known price $B_0$, the yield is the solution y to the equation

$$B_0\sum_{t=1}^{T}\frac{C_t}{(1+y)^t}$$

This calculation therefore has the same qualifications as discussed earlier calculating IRR, it supposes
reinvestment of coupon at the bond yield (the IRR).

There is much less likelihood we’ll have technical problems with multiple solutions when doing this yield
estimation for bonds, since the structure of cash flows is such that there exist only one solution to the
equation. The algorithm for finding a bonds yield to maturity shown in **Code 4.2** is thus simple
bisection. We know that the bond yield is above zero and set zero as a lower bound on the bond yield.
We then find an upper bound on the yield by increasing the interest rate until the bond price with this
interest rate is negative. We then bisect the interval between the upper and lower until we are “close
enough.” **Code 4.2** implements this idea.

In [4]:
# Code 4.2: Bond yield calculation with discrete, annual compounding

def bond_yield_to_maturity_discrete(times, cashflows, bondprice):
    
    ACCURACY = 1e-5
    MAX_ITERATIONS = 200
    bot = 0
    top = 1.
    
    while bonds_price_discrete(times, cashflows, top) > bondprice: 
        top = top * 2
        
    r = .5 * (top + bot)
    
    for _ in range(MAX_ITERATIONS):
        diff = bonds_price_discrete(times, cashflows, r) - bondprice
        
        if np.abs(diff) < ACCURACY:
            return r
        
        if diff > 0:
            bot = r
            
        else:
            top = r
            
        r = .5 * (top + bot)
        
    return r

#### Example 
A 3 year bond with a face value of $100 makes annual coupon payments of 10%. The current interest rate
(with annual compounding) is 9%.

1. Find the bond’s current price.
2. Find the bond’s yield to maturity.

In [5]:
c = np.array([10, 10, 110])
t = np.arange(1, 4)
r = .09
b = np.sum(c * (1. / np.power((1 + r), t)))

In [6]:
print('Bond price, 9 percent discretely compounded interest = {:.3f}'.format(
        bonds_price_discrete(t, c, r)))

print('bond yield to maturity = {:.2f}'.format(bond_yield_to_maturity_discrete(t, c, b)))

Bond price, 9 percent discretely compounded interest = 102.531
bond yield to maturity = 0.09


### 4.1.3 Duration

When holding a bond one would like to know how sensitive the value of the bond is to changes in
economic environment. The most relevent piece of the economic environment is the current interest
rate. An important component of such calculation is the duration of a bond. The duration of a bond
should be interpreted as the weighted average maturity of the bond, and is calculated as

$$Duration = \frac{\sum_{t}t\frac{C_t}{(1+r)^t}}{Bond\ Price'}$$

where $C_t$ is the cash flow in period $t$, and $r$ the interest rate. Using the bond price calculated in equation 4.1 we calculate duration as

$$D=\frac{\sum_{t}\frac{tC_t}{(1+r)^t}}{\sum_{t}\frac{C_t}{((1+r)^t}}$$

In [7]:
# Code 4.3: Bond duration using discrete, annual compounding and a flat term structure

def bonds_duration_discrete(times, cashflows, r):
    
    b = 0
    d = 0
    
    for i in range(len(times)):
        
        d += times[i] * cashflows[i] / np.power((1 + r), times[i])
        b += cashflows[i] / np.power((1 + r), times[i])
        
    return d / b

An alternative approach to calculating duration is calculate the yield to maturity $y$ for the bond, and
use that in estimating the bond price. This is called *Macaulay Duration*. First one calculates $y$, the
yield to maturity, from

$$Bond\ price = \sum_{t=1}^{T}\frac{C_t}{(1+y)^t}$$

and then use this y in the duration calculation:

$$ Macaulay\ duration=\frac{\sum_{t}\frac{tC_t}{(1+r)^t}}{\sum_{t}\frac{C_t}{((1+r)^t}}$$

Note though, that in the present case, with a flat term structure, these should produce the same number.
If the bond is priced correctly, the yield to maturity must equal the current interest rate. If $r = y$ the
two calculations in equations (4.4) and (4.5) obviously produces the same number.

#### Example
A 3 year bond with a face value of $100 makes annual coupon payments of 10%. The current interest rate
(with annual compounding) is 9%.

In [8]:
# Code 4.4: Calculating the Macaulay duration of a bond

def bonds_duration_macaulay_discrete(times, cashflows, bondprice):
    y = bond_yield_to_maturity_discrete(times, cashflows, bondprice)
    
    return bonds_duration_discrete(times, cashflows, y) # use YTM in duration calculation

1. Determine the current bond price.
2. Calculate the duration using the current interest rate.
3. Calculate the duration using the MaCaulay definition.

Need to calculate the following:

The current bond price: $B_0=\frac{10}{(1+0.09)^1}+\frac{10}{(1+0.09)^2}+\frac{110}{(1+0.09)^3}=102.531$

The bond's duration: $D=\frac{1}{102.531}\Big(\frac{1.10}{1.09}+\frac{2.10}{1.09^2}+\frac{3.110}{1.09^3}\Big)=2.74$

In [9]:
c = np.array([10., 10, 110])
t = np.arange(1, 4)
r = .09
b = np.sum(c * (1. / np.power((1 + r), t)))
d = np.sum(( 1 / b) * t * c * (1. / np.power((1 + r), t)))
y = np.irr(np.insert(c, 0, -b))
dm = np.sum((1 / b) * t * c * (1. / np.power((1 + r), t)))

In [10]:
print('bonds price = {:.3f}'.format(bonds_price_discrete(t, c, r)))
print('bond duration = {:.5f}'.format(bonds_duration_discrete(t, c, r)))
print('bond macaulay = {:.5f}'.format(bonds_duration_macaulay_discrete(t, c, b)))

bonds price = 102.531
bond duration = 2.73895
bond macaulay = 2.73895


### 4.1.4 Measuring bond sensitivity to interest rate changes

Now, the reason for why we say that we can measure the sensitivity of a bond price using duration. To
a first approximation, $\Delta B_0$, the change in the bond price for a small change in the interest rate $\Delta r$, can
be calculated

$$\frac{\Delta B_0}{B_0} \approx - \frac{D}{1+r} \Delta r$$

where $D$ is the bond’s duration. For simplicity one often calculates the term in front of the $\Delta y$ in the above, $\frac{D}{1+y}$ directly and terms it the bond’s *modified duration*.

$$Modified\ Duration=D^*=\frac{D}{1+r}$$

The sensitivity calculation is then
$$D^*=\frac{D}{1+y}$$

In [11]:
# Code 4.5: Modified duration
def bonds_duration_modifed_discrete(times, cashflows, bond_price):
    
    y = bond_yield_to_maturity_discrete(times, cashflows, bond_price)
    d = bonds_duration_discrete(times, cashflows, y)
    
    return d / (1 + y)

The modified duration measures the angle of the tangent at the current bond yield. Approximating
the change in bond price with duration is thus only a first order approximation. To improve on this
approximation we also need to account for the curvature in the relationship between bond price and
interest rate. To quantify this curvature we calculate the *convexity* of a bond.

$$Convexity=Cx=\frac{1}{B_0}\frac{1}{((1+r)^2}\sum_{t=1}^{T}(t+t^2)\frac{C_t}{(1+r)^t}$$

This calculation is implemented in **Code 4.6**. To improve on the estimate of how the bond price

In [12]:
# Code 4.6: Bond convexity with a flat term structure and annual compounding

def bonds_convexity_discrete(times, cashflows, r):
    
    cx = 0.
    
    for i in range(len(times)):
        cx += cashflows[i] * times[i] * (times[i] + 1) / np.power((1 + r), times[i])
    
    b = bonds_price_discrete(times, cashflows, r)

    return (cx / (np.power((1 + r), 2))) / b

change when the interest rates changes you will then calculate

$$\frac{\Delta B_0}{B_0}\approx-D^*\Delta y + \frac{1}{2}Cx(\Delta y)^2$$

#### Example 
A 3 year bond with a face value of $100 makes annual coupon payments of 10%. The current interest rate
(with annual compounding) is 9%.
1. Determine the current bond price.
2. Suppose the interest rate changes to 10%, determine the new price of the bond by direct calculation.
3. Use duration to estimate the new price and compare it to the correct price.
4. Use convexity to improve on the estimate using duration only.

In [13]:
c = np.array([10., 10, 110])
t = np.arange(1, 4)
r = .09
b = np.sum(c * (1. / np.power((1 + r), t)))
d = np.sum(( 1 / b) * t * c * (1. / np.power((1 + r), t)))

In [14]:
print('bonds price = {:.3f}'.format(bonds_price_discrete(t, c, r)))
print('bond duration = {:.5f}'.format(bonds_duration_discrete(t, c, r)))
print('bond macaulay = {:.5f}'.format(bonds_duration_modifed_discrete(t, c, b)))
print('bond convexity = {:.5f}'.format(bonds_convexity_discrete(t, c, r)))
print('new bond price = {:.0f}'.format(bonds_price_discrete(t, c, .1)))

bonds price = 102.531
bond duration = 2.73895
bond macaulay = 2.51280
bond convexity = 8.93248
new bond price = 100


### 4.2 Continously compounded interest

Some important differences is worth pointing out. When using continously compounded interest, one
does not need the concept of *modified duration*. In the continously compounded case one uses the
calculated duration directly to approximate bond changes, as seen in the formulas describing the approximation of bond price changes. Note also the difference in the *convexity* calculation, one does not divide by $(1 + y)^2$ in the continously compounded formula, as was done in the discrete case.

In [15]:
# Code 4.7: Bond price calculation with continously compounded interest and a flat term structure
def bonds_price(cashflows_times, cashflows, r):
    
    p = 0
    
    for i in range(len(cashflows_times)):
        p += np.exp(-r * cashflows_times[i]) * cashflows[i]
        
    return p

In [16]:
# Code 4.8: Bond duration calculation with continously compounded interest and a flat term strucutre
def bonds_duration(cashflows_times, cashflows, r):
    
    s = 0
    d1 = 0
    
    for i in range(len(cashflows_times)):
        s += cashflows[i] * np.exp(-r * cashflows_times[i])
        d1 += cashflows_times[i] * cashflows[i] * np.exp(-r * cashflows_times[i])
        
    return d1 / s

In [17]:
# Code 4.9: Calculating the Macaulay duration of a bond with continously
# compounded interest and a flat term structure
def bonds_duration_macaulay(cashflows_times, cashflows, bond_price):
    
    y = bond_yield_to_maturity_discrete(cashflows_times, cashflows, bond_price)
    
    return bonds_duration(cashflows_times, cashflows, y) # use YTM in duration

In [18]:
# Code 4.10: Bond convexity calculation with continously compounded interest
# and a flat term structure
def bonds_convexity(times, cashflows, r):
    
    c = 0
    for i in range(len(times)):
        
        c += cashflows[i] * np.power(times[i], 2) * np.exp(-r * times[i])
        
    b = bonds_price(times, cashflows, r)
    
    return c / b

#### Example
A 3 year bond with a face value of $100 makes annual coupon payments of 10%. The current interest rate
(with continous compounding) is 9%.
1. Calculate the bond’s price, yield to maturity, duration and convexity.
2. Suppose the interest rate falls to 8%. Estimate the new bond price using duration, and compare with
the actual bond price using the correct interest rate.

In [19]:
c = np.array([10., 10, 110])
t = np.arange(1, 4)
r = .09

In [20]:
print('bonds price = {:.3f}'.format(bonds_price(t, c, r)))
print('bond duration = {:.5f}'.format(bonds_duration(t, c, r)))
print('bond convexity = {:.5f}'.format(bonds_convexity(t, c, r)))
print('new bond price = {:.3f}'.format(bonds_price(t, c, .08)))

bonds price = 101.464
bond duration = 2.73753
bond convexity = 7.86779
new bond price = 104.282


The term structure is flat, and compounding is continous. Consider two definitions of duration, the “usual” definition $D = \frac{1}{B_0}\sum_{i}t_iC_{t_i}e^{-rt}$ and the Macaulay defintion: $D=\frac{1}{b_0}\sum_{i}t_iC_{t_i}e^{-yt_i}$, where $B_0$ is the current bond price, $C_{t_i}$ is the coupon payment at date $t_i$, $r$ is the current interest rate and $y$ is the bond’s yield to maturity.